In [10]:
import sys
import os

sys.path.append('../../')

from datetime import date, datetime
r_today = date.today().strftime("%y%m%d")
r_time = datetime.now().strftime("%H%M")
# dataset_seed = int(f'{r_today}{r_time}')
dataset_seed = 2310091953

from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop

from src.hypermodels import *
from src.utils  import *
from src.dataset import *

question_id = 2

In [11]:
image_size = 224

# duel_results_path = f'../../data/duel_results/duels_question_{question_id}'
# image_file_dir = '../../data/Mapillary_data/Sample_web_green'
# 
# print(f'Dataset Seed:{dataset_seed}') # For reproducibility, seed is defined using the date and time (e.g. 2021-01-01 12:30:15 -> seed = 20210101123015)

# (x_train, y_train), (x_val, y_val), (x_test, y_test) = prepare_dataset(duel_results_path, 
#                                                                        image_file_dir, 
#                                                                        img_size=image_size,
#                                                                        model_type='ranking',
#                                                                        shuffle=True, 
#                                                                        shuffle_seed=dataset_seed)

# Load from the saved numpy arrays
x1_train,x2_train = np.load('../../data/Q2_Datasets_RandomShuffled_1/x_train.npy')
x1_val,x2_val = np.load('../../data/Q2_Datasets_RandomShuffled_1/x_val.npy')
x1_test,x2_test = np.load('../../data/Q2_Datasets_RandomShuffled_1/x_test.npy')

y_train = np.load('../../data/Q2_Datasets_RandomShuffled_1/y_train.npy')
y_val = np.load('../../data/Q2_Datasets_RandomShuffled_1/y_val.npy')
y_test = np.load('../../data/Q2_Datasets_RandomShuffled_1/y_test.npy')

x_train = [x1_train,x2_train]
x_val = [x1_val,x2_val]
del x1_train,x2_train,x1_val,x2_val

In [15]:
from keras.models import Model, load_model, model_from_json

# build the model using model architecture
saved_model_dir = '../../trainings/q2_model_training/hypermodel_tuning/BayesianOptimization_2310101054/Model_8_Ranking_VGG19_BestValAcc0.7338'
model_file_path = '../../trainings/q2_model_training/hypermodel_tuning/BayesianOptimization_2310101054/Model_8_Ranking_VGG19_BestValAcc0.7338/Siamese_Ranking_Network.keras'
model_architecture_file_path = '../../trainings/q2_model_training/hypermodel_tuning/BayesianOptimization_2310101054/best_model_architecture_8.json'

trained_model = load_model(model_file_path, compile=True)

# load json and create model
json_file = open(model_architecture_file_path, 'r')
loaded_model_json = json_file.read()
json_file.close()

untrained_model =  model_from_json(loaded_model_json)
untrained_model.summary()

Model: "Siamese_Ranking_Network"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_26 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_27 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Ranking_Network (Functional)   (None, 1)            22437153    ['input_26[0][0]',               
                                                                  'input_27[

In [ ]:
# Train the model
optimizers = {
    'adam': Adam(learning_rate=0.00001),
    'rmsprop': RMSprop(learning_rate=0.00001)
}

untrained_model.compile(loss='binary_crossentropy', optimizer=optimizers['rmsprop'], metrics=['accuracy'])

model = untrained_model
# model = trained_model

history = model.fit(
    x_train, 
    y_train,
    batch_size=64,
    epochs=50,
    validation_data=(x_val, y_val),
    callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='auto', restore_best_weights=True)]
)

Epoch 1/50
41/42 [============================>.] - ETA: 0s - loss: 0.7275 - accuracy: 0.5743

In [0]:
from src.utils import safe_save_training_results

best_val_acc = max(history.history['val_accuracy'])
save_dir = os.path.join(saved_model_dir, f'Retrain_BestValAcc{best_val_acc}')

safe_save_training_results(hyperparams_dict=None,
                           model=model,
                           model_save_type='keras', 
                           history=history, 
                           his_save_type='csv', 
                           save_dir = save_dir,)

# Save the plot
plot_model_metrics(history, ['loss', 'accuracy'], save_dir=save_dir)



In [ ]:
import keras.backend as K
K.clear_session()